# Variational Graph AutoEnconders

How it works?
Just like auto encoders but with a twist. Rather than representing the embedding space as a position vector, we represent it as a multivariate Gaussian.

So usually the mapping of an auto encoder goes from input channel I -> embedding vector of size L. So the mapping is simply I -> L

In a VAE another parallel branch is made to capture the sigma / variance of the original position vector. So the mapping goes from I -> L (mu) + L (sigma).

These two layers are reparametrized so that back propagation can work and hence have a single VAE. 

Just like in graphs u have the autoencoder encode every node feature, here the VGAE will encode all the node features as well.


In [1]:
from torch_geometric.nn import VGAE
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [2]:
dataset = Planetoid('data','CiteSeer',transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)

class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self,in_channels,out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels,512,cached=True)
        self.conv1_5 = GCNConv(512,128,cached=True)         # cached only for transductive learning
        self.conv_mu = GCNConv(128,out_channels,cached=True)
        self.conv_logstd = GCNConv(128,out_channels,cached=True)
        
    def forward(self, x, edge_index):
        x = self.conv1(x,edge_index).relu()
        x = self.conv1_5(x,edge_index).relu()
        return self.conv_mu(x,edge_index), self.conv_logstd(x,edge_index)

C:\Users\msadi\anaconda3\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [3]:
out_channels = 32
num_features = dataset.num_features
epochs = 1000

model = VGAE(VariationalGCNEncoder(num_features,out_channels))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [4]:
def train(model, optimizer, x, pos_edge):
    model.train()
    optimizer.zero_grad()
    z = model.encode(x,pos_edge)
    loss = model.recon_loss(z, pos_edge)
    # Loss = Expected Loss + KL_Divergence Loss
    loss = loss + (1/data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

def test(pos_edge,neg_edge,x,train_pos_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x,train_pos_index)
    return model.test(z,pos_edge,neg_edge)

In [5]:
from torch.utils.tensorboard import SummaryWriter

In [6]:
writer = SummaryWriter('runs/VGAE_experiment_'+'32d_1000_epochs')

In [7]:

for epoch in range(1,epochs+1):
    loss = train(model,optimizer,x,train_pos_edge)   
    auc,ap = test(data.test_pos_edge_index,data.test_neg_edge_index,x,train_pos_edge)
    if epoch%100==0:
        print("Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}".format(epoch,auc,ap))
    writer.add_scalar('auc train',auc,epoch)
    writer.add_scalar('ap train',ap,epoch)

Epoch: 100, AUC: 0.8477, AP: 0.8615
Epoch: 200, AUC: 0.8624, AP: 0.8774
Epoch: 300, AUC: 0.8769, AP: 0.8905
Epoch: 400, AUC: 0.8849, AP: 0.8980
Epoch: 500, AUC: 0.8729, AP: 0.8918
Epoch: 600, AUC: 0.8678, AP: 0.8918
Epoch: 700, AUC: 0.8686, AP: 0.8908
Epoch: 800, AUC: 0.8673, AP: 0.8888
Epoch: 900, AUC: 0.8662, AP: 0.8883
Epoch: 1000, AUC: 0.8661, AP: 0.8895
